In [ ]:
{-# LANGUAGE FunctionalDependencies #-}
{-# LANGUAGE MonomorphismRestriction #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE ConstraintKinds #-}
{-# LANGUAGE TemplateHaskell #-}
{-# LANGUAGE OverloadedStrings #-}

import Control.Arrow
import Control.Monad.State
import Control.Monad.Trans.Class
import Numeric.LinearAlgebra
import Numeric.LinearAlgebra.Data
import Numeric
import Prelude hiding ((<>))
import Network.HTTP.Client
import Network.HTTP.Types.Status (statusCode)
import qualified Codec.Compression.GZip as GZip
import qualified Data.ByteString.Lazy as B
import System.Directory
import System.FilePath.Posix

\begin{eqnarray}
Sigmoid \nonumber \\
  h(x) &=& \frac
  {1}
  {1 + \exp(-x)} \\
\nonumber \\
ReLU \nonumber \\
  h(x) &=& \begin{cases}
  x & (x > 0) \\
  0 & (x \le 0)
  \end{cases}
\end{eqnarray}

In [ ]:
sigmoid :: (Floating a) => a -> a
sigmoid a = fromIntegral 1 / (fromIntegral 1 + exp (-a))

sigmoidm :: (Floating a, Container Matrix a) => Matrix a -> Matrix a
sigmoidm = cmap sigmoid

sigmoidBackward :: (Floating a, Num (Vector a), Container Matrix a) => Matrix a -> Matrix a -> Matrix a
sigmoidBackward y d = d * (cmap (\a -> fromIntegral 1 - a) y) * y

relu :: (Ord a, Num a) => a -> a
relu = max $ fromIntegral 0

relum :: (Ord a, Num a, Container Matrix a) => Matrix a -> Matrix a
relum = cmap relu

relumBackward :: (Ord a, Num a, Num (Matrix a), Container Vector a, Container Matrix a) => Matrix a -> Matrix a -> Matrix a
relumBackward x d = d * mask
    where mask = cmap (\a -> fromIntegral $ if (0 < a) then 1 else 0) x

\begin{eqnarray}
Cross Entropy \nonumber \\
  L &=& -\displaystyle \sum_{i=1}^{n}t_i \log (y_i + d) \\
  t &:& 教師ラベル \nonumber \\
  d &:& 微小値 \nonumber \\
 \nonumber \\
Softmax \nonumber \\
  y_k &=& \frac
  {\exp(a_k - \hat{a})}
  {\displaystyle \sum_{i=1}^{n}\exp(a_i - \hat{a})} \\
  \hat{a} &=& \max \{ a_{1...n} \} \nonumber
\end{eqnarray}


In [ ]:
softmax :: (Floating a, Container Vector a) => Vector a -> Vector a
softmax v = cmap (/s) v'
    where
    m = maxElement v
    v' = cmap (\a -> exp (a - m)) v
    s = sumElements v'

softmaxm :: (Floating a, Container Vector a) => Matrix a -> Matrix a
softmaxm m = fromRows $ map softmax $ toRows m

crossEntropy :: (Floating a, Num (Vector a), Container Vector a) => Vector a -> Vector a -> a
crossEntropy t y = -(sumElements $ cmap log (y + d) * t)
    where d = 1e-10

crossEntropym :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> a
crossEntropym t m = sum vs / batchSize
    where 
    vs = uncurry crossEntropy `map` (toRows t `zip` toRows m)
    batchSize = fromIntegral $ rows t

softmaxWithCross :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> a
softmaxWithCross t = crossEntropym t . softmaxm

softmaxWithCrossBackward :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> Matrix a
softmaxWithCrossBackward t y = (y - t) / batchSize
    where batchSize = fromIntegral $ rows t

In [ ]:
affinem :: (Floating a, Numeric a, Num (Vector a)) => Matrix a -> Vector a -> Matrix a -> Matrix a
affinem w b x = x <> w + b'
    where b' = fromColumns $ replicate (rows x) b

affinemBackward :: (Floating a, Numeric a) => Matrix a -> Matrix a -> Matrix a -> (Matrix a, Matrix a, Vector a)
affinemBackward w x d = (dx, dw, db)
    where
    dx = d <> tr w
    dw = tr x <> d
    db = fromList $ sumElements `map` toColumns d

In [ ]:
type Weight a = Matrix a
type Bias a = Vector a
type SignalX a = Matrix a
type SignalY a = Matrix a
type Diff a = Matrix a
type TeacherBatch a = Matrix a
type InputBatch a = Matrix a
newtype TrainBatch a = TrainBatch (TeacherBatch a, InputBatch a)

In [ ]:
data ForwardLayer a =
    AffineForward (Weight a) (Bias a)
  | SigmoidForward
  | ReLUForward
  | JoinedForwardLayer (ForwardLayer a) (ForwardLayer a)
  deriving (Show)

infixl 4 ~>
(~>) :: ForwardLayer a -> ForwardLayer a -> ForwardLayer a
a ~> (JoinedForwardLayer x y) = (a ~> x) ~> y
a ~> b = JoinedForwardLayer a b

data OutputLayer a = SoftmaxWithCrossForward
  deriving (Show)

data ForwardNN a = ForwardNN (ForwardLayer a) (OutputLayer a)
  deriving (Show)

data BackwardLayer a = 
    AffineBackward (Weight a) (Bias a) (SignalX a)
  | SigmoidBackward (SignalY a)
  | ReLUBackward (SignalX a)
  | JoinedBackwardLayer (BackwardLayer a) (BackwardLayer a)
  deriving (Show)

infixr 4 <~
(<~) :: BackwardLayer a -> BackwardLayer a -> BackwardLayer a
(JoinedBackwardLayer x y) <~ b = x <~ (y <~ b)
a <~ b = JoinedBackwardLayer a b

data BackputLayer a = SoftmaxWithCrossBackward (TeacherBatch a) (SignalY a)

data BackwardNN a = BackwardNN (BackwardLayer a) (BackputLayer a)

type NElement a = (Ord a, Floating a, Numeric a, Num (Vector a))

In [ ]:
forward :: NElement a => ForwardLayer a -> SignalX a -> (BackwardLayer a, SignalY a)
forward (AffineForward w b) x = (AffineBackward w b x, affinem w b x)
forward SigmoidForward x = let y = sigmoidm x in (SigmoidBackward y, y)
forward ReLUForward x = (ReLUBackward x, relum x)
forward (JoinedForwardLayer a b) x0 = (a' <~ b', x2)
    where
    (a', x1) = forward a x0
    (b', x2) = forward b x1

backward :: NElement a => BackwardLayer a -> Diff a -> (ForwardLayer a, Diff a)
backward (AffineBackward w b x) d =  (AffineForward (w + w') (b + b'), x')
    where (x', w', b') = affinemBackward w x d
backward (SigmoidBackward y) d = (SigmoidForward, sigmoidBackward y d)
backward (ReLUBackward x) d = (ReLUForward, relumBackward x d)
backward (JoinedBackwardLayer a b) d0 = (a' ~> b', d2)
    where
    (b', d1) = backward b d0
    (a', d2) = backward a d1

output :: NElement a => OutputLayer a -> TeacherBatch a -> SignalY a -> (BackputLayer a, a)
output SoftmaxWithCrossForward t y = (SoftmaxWithCrossBackward t y, softmaxWithCross t y)

backput :: NElement a => BackputLayer a -> (OutputLayer a, Diff a)
backput (SoftmaxWithCrossBackward t y) = (SoftmaxWithCrossForward, softmaxWithCrossBackward t y)

In [ ]:
learnForward :: NElement a => ForwardNN a -> TrainBatch a -> (BackwardNN a, a)
learnForward (ForwardNN layers loss) (TrainBatch (t, x)) = (BackwardNN layers' loss', result)
    where
    (layers', y) = forward layers x
    (loss', result) = output loss t y

learnBackward :: NElement a => BackwardNN a -> ForwardNN a
learnBackward  (BackwardNN layers loss) = ForwardNN layers' loss'
    where
    (loss', d) = backput loss
    (layers', _) = backward layers d

learn :: NElement a => ForwardNN a -> TrainBatch a -> (ForwardNN a, a)
learn a = first learnBackward . learnForward a

learnAll :: NElement a => ForwardNN a -> [TrainBatch a] -> (ForwardNN a, [a])
learnAll origin = foldr f (origin, [])
    where f p (a, results) = second (: results) $ learn a p

predict :: NElement a => ForwardLayer a -> Vector a -> Int
predict layers = maxIndex . flatten . snd . (forward layers) . asRow 

In [ ]:
a = SigmoidForward
a ~> (a ~> a) ~> a
b = SigmoidBackward (matrix 1 [0])
b <~ (b <~ b) <~ b

In [ ]:
rootDir = "mnist"
urlBase = "http://yann.lecun.com/exdb/mnist/"
keyFiles = [
    ("train_img", "train-images-idx3-ubyte.gz")
  , ("train_label", "train-labels-idx1-ubyte.gz")
  , ("test_img", "t10k-images-idx3-ubyte.gz")
  , ("test_label", "t10k-labels-idx1-ubyte.gz")
  ]

downloadFile :: Manager -> String -> FilePath -> IO ()
downloadFile manager url file = do
    e <- doesFileExist file
    if e then return () else do
        request <- parseRequest url
        response <- httpLbs request manager
        let bs = GZip.decompress $ responseBody response
        B.writeFile file bs

saveAll :: Manager -> IO [()]
saveAll manager = do
    mapM saveFile keyFiles
    where
    saveFile (key, name) = do
        let url = urlBase ++ name
        let file = rootDir </> key
        downloadFile manager url file

startDownload = newManager defaultManagerSettings >>= saveAll